## Newsletter Agent의 노드 구축하기

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### 사용자가 입력한 키워드에 대한 뉴스 검색 함수

In [1]:
from tavily import TavilyClient
from langchain_core.tools import tool

In [2]:
tavily_client = TavilyClient()

def search_recent_news(keyword):
    """
    This tool interacts with the Tavily AI API to search for recent news articles related to a given keyword.

    Args:
        keyword (str): The keyword or phrase to search for in the news articles.

    Returns:
        list: 
        A list of titles, each containing up to 10 of the most recent news articles related to the keyword.
        - 'title' (str): The title of the news article.

    Example:
        response = search_news("OpenAI")
        # Returns a list of news articles published in the last day related to OpenAI.
    """
    article_info = []
    
    # Making the request to the Tavily API
    response = tavily_client.search(
        query=keyword, 
        max_results=10, 
        topic="news", 
        days=7
    )
    title_list = [i['title'] for i in response['results']]
    return title_list

In [3]:
keyword = "OpenAI"
result = search_recent_news(keyword)
result

['OpenAI is making its own browser, presumably to spite Google and cut out the middleman when collecting user data - PC Gamer',
 'OpenAI Hits Pause on Open-Source Model: Safety First! - OpenTools',
 'OpenAI delays the release of its open model, again - TechCrunch',
 '$300 billion, 500 million users, and no time to enjoy it: The sharks are circling OpenAI - Business Insider Africa',
 "Microsoft's Existing OpenAI Deal May Be Undercutting Ad Agency Partnerships - Adweek",
 'OpenAI tightens the screws on security to keep away prying eyes - TechCrunch',
 'Google scuppers $3bn OpenAI deal by buying key technology and developers - Gagadget.com',
 'Sam Altman says OpenAI is delaying its open-weight model to run extra safety tests - Business Insider',
 'ChatGPT keeps having more memories of you - Axios',
 'OpenAI’s Windsurf deal is off — and Windsurf’s CEO is going to Google - The Verge']

### 뉴스 테마 설정 함수

In [4]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

# Data model
class NewsletterThemeOutput(BaseModel):
    """Output model for structured theme and sub-theme generation."""

    theme: str = Field(
        description="The main newsletter theme based on the provided article titles."
    )
    sub_themes: list[str] = Field(
        description="List of sub-themes or key news items to investigate under the main theme, ensuring they are specific and researchable."
    )


# LLM with function call
structured_llm_newsletter = llm.with_structured_output(NewsletterThemeOutput)

# Prompt
system = """
You are an expert helping to create a newsletter. 
Based on a list of article titles provided, your task is to choose a single, 
specific newsletter theme framed as a clear, detailed question that grabs the reader's attention. 

In addition, generate 5 sub-themes that are highly specific, researchable news items or insights under the main theme. 
Ensure these sub-themes reflect the latest trends in the field and frame them as compelling news topics.

The output should be formatted as:
- Main theme (in question form)
- 3-5 sub-themes (detailed and focused on emerging trends, technologies, or insights).

The sub-themes should create a clear direction for the newsletter, avoiding broad, generic topics.
All your output should be in Korean

"""

# This is the template that will feed into the structured LLM
theme_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Article titles: \n\n {article_titles}"),
    ]
)

# Chain together the system prompt and the structured output model
newsletter_generator = theme_prompt | structured_llm_newsletter

In [6]:
output = newsletter_generator.invoke({"article_titles":result})
subthemes = output.sub_themes
subthemes

['OpenAI의 독자 브라우저 개발: Google에 대한 반격과 사용자 데이터 수집의 직접화',
 '안전성을 최우선 과제로 삼은 OpenAI의 오픈소스 모델 지연 이유 분석',
 'OpenAI가 마주한 시장의 압박: 3000억 달러, 5억 사용자의 미래는?',
 'Microsoft의 OpenAI와의 협약이 광고 대행사 파트너십에 미치는 영향',
 'OpenAI의 보안 강화 조치: 외부의 위협으로부터 어떻게 방어하고 있는가?']

In [7]:
from typing import List

def subtheme_generator(recent_news: List[str]):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

    # Data model
    class NewsletterThemeOutput(BaseModel):
        """Output model for structured theme and sub-theme generation."""

        theme: str = Field(
            description="The main newsletter theme based on the provided article titles."
        )
        sub_themes: list[str] = Field(
            description="List of sub-themes or key news items to investigate under the main theme, ensuring they are specific and researchable."
        )
    # LLM with function call
    structured_llm_newsletter = llm.with_structured_output(NewsletterThemeOutput)

    # Prompt
    system = """
    You are an expert helping to create a newsletter. Based on a list of article titles provided, your task is to choose a single, 
    specific newsletter theme framed as a clear, detailed question that grabs the reader's attention. 

    In addition, generate 3 to 5 sub-themes that are highly specific, researchable news items or insights under the main theme. 
    Ensure these sub-themes reflect the latest trends in the field and frame them as compelling news topics.

    The output should be formatted as:
    - Main theme (in question form)
    - 3-5 sub-themes (detailed and focused on emerging trends, technologies, or insights).

    The sub-themes should create a clear direction for the newsletter, avoiding broad, generic topics.
    All your output should be in Korean

    """

    # This is the template that will feed into the structured LLM
    theme_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "Article titles: \n\n {recent_news}"),
        ]
    )

    # Chain together the system prompt and the structured output model
    subtheme_chain= theme_prompt | structured_llm_newsletter
    output = subtheme_chain.invoke({"recent_news":recent_news})
    return output

In [8]:
output = subtheme_generator({"article_titles":result})
subthemes = output.sub_themes
output

NewsletterThemeOutput(theme='OpenAI의 현재 상황은 무엇이며, 이를 통해 우리가 배울 수 있는 점은 무엇인가?', sub_themes=['OpenAI가 자사 브라우저를 개발하는 이유: 구글과의 경쟁 구도 분석', 'OpenAI의 오픈 소스 모델 지연, 안전성 우선 과제가 드러난 배경', 'OpenAI와 마이크로소프트의 파트너십이 광고 산업에 미치는 영향', 'OpenAI의 보안 강화 조치: 사용자 데이터 보호를 위한 새로운 전략', 'OpenAI의 전략적 결정 변화: Windsurf 거래 중단의 의미와 미래 전망'])